# Brainstorm Proyecto 1 - DP

Se plantean dos posibles enfoques. 

1. Uno donde se parametrizan en una recurrencia el movimiento de Indiana, Marion y Salah (Dicha recurrencia será denomoniada $P$)
2. Un conjunto de tres recurrencias que modelan el comportamiento de cada personaje y que son invocadas de acuerdo a distintas prioridades de recorrido para cada una de las personas.

Antes de eso, se definen las siguientes entradas y salidas

| E/S | Nombre | Tipo | Descripción |
|-|-|-|-|
|E| A | Array[0,R)[0,C) of **int** | Matriz con la representación de la cantidad de reliquias y maldiciones de la pirámide|
|S| s | **int** | Cantidad máxima de reliquias que pueden obtener los tres personajes |

## Enfoque de única recurrencia

Tanto Marion, como Indiana y Salah pueden moverse en un espacio de $C \times \left\lfloor \dfrac{R}{2} \right\rfloor$. Por consiguiente, una estructura que pueda almacenar la cantidad maxima de reliquias obtenidas para cada combinacion de posiciones deberá tener una forma de $C \times C \times C \times \left\lfloor \dfrac{R}{2} \right\rfloor$

Para este enfoque tendremos en cuenta los siguientes puntos:

1. Todas las casillas invalidas deberán tener un valor de $-\infty$ para aquellas configuraciones invalidas (Ejemplo: Aquellas casillas con maldiciones y aquellas que en la primera y ultima fila que no correspondan al punto de partida de los personajes) 
2. La casilla inicial de cada personaje debe iniciar con cero reliquias.
3. Los demás casos recursivos representan todas las cobinaciones posibles de los movimientos de los personajes

$$ P(i,m,s,r) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r = 0 \wedge i=0 \wedge m=C-1 \wedge s = \left\lfloor \frac{C}{2} \right\rfloor \\
        -\infty & & \text{si} & & A[r][i]=-1 \vee A[r][m]=-1 \vee A[r][s]=-1 \vee \left(r=0 \wedge i>0  \wedge m < C-1 \wedge s \neq \left\lfloor \frac{C}{2} \right\rfloor \right)\\
        ... 
    \end{array}
    \right.
$$

**¿Que problema tiene?** Los indices i,m,s van los tres desde $0$ hasta $C-1$ y el indice r va desde $0$ hasta $\left\lfloor \dfrac{R}{2} \right\rfloor$. Esto significa que la complejidad tanto temporal como espacial de este algoritmo será de orden $O(C^3 R)$. Lo cual se convierte en un problema muy costoso computacionalmente hablando

## Enfoque de muchas recurrencias

Para este enfoque, diseñaremos una recurrencia para cada personaje. $I$ estara asociada a Indiana, $M$ estara asociada a Marion y $S$ estara asociada a Salah

$$ I(r,i) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r=0 \wedge i = 0 \\
        -\infty & & \text{si} & &  A[r][i] = -1 \vee (r=0 \wedge i > 0) \\
        \max (I(r-1,i),I(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge i = 0 \\
        \max (I(r-1,i),I(r-1,i-1)) + A[r][i] & & \text{si} & & r>0 \wedge i = C-1 \\
        \max (I(r-1,i),I(r-1,i-1),I(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge 0 < i < C-1 \\
    \end{array}
    \right.
$$

$$ M(r,i) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r=0 \wedge i = C-1 \\
        -\infty & & \text{si} & &  A[r][i] = -1 \vee (r=0 \wedge i < C-1) \\
        \max (M(r-1,i),M(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge i = 0 \\
        \max (M(r-1,i),I(r-1,i-1)) + A[r][i] & & \text{si} & & r>0 \wedge i = C-1 \\
        \max (M(r-1,i),I(r-1,i-1),M(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge 0 < i < C-1 \\
    \end{array}
    \right.
$$

$$ S(r,i) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r= R-1 \wedge i = \lfloor \frac{C}{2} \rfloor\\
        -\infty & & \text{si} & &  A[r][i] = -1 \vee (r = R-1 \wedge i \neq \lfloor \frac{C}{2} \rfloor) \\
        \max(S(r+1,i),S(r+1,i+1)) & & \text{si} & & r < R-1 \wedge i = 0 \\
        \max(S(r+1,i),S(r+1,i-1)) & & \text{si} & & r < R-1 \wedge i = C-1 \\
        \max(S(r+1,i),S(r+1,i-1),S(r+1,i+1)) & & \text{si} & & r < R-1 \wedge 0 < i < C-1 \\
    \end{array}
    \right.
$$

Probamos los ejemplos definidos en el documento

In [1]:
from ProblemaP1Tests import *
from FullDP import *

A =  [
        [0,9,1,10,0],
        [-1,5,5,25,5],
        [1,5,1,5,7],
        [5,5,5,15,2],
        [55,3,0,4,1]
    ]

print(f'Enfoque de varias recurrencias: {solvePathsProblem(A)}')
print(f'Enfoque de unica recurrencia: {fullSolutionDP(A)[0]}')

Enfoque de varias recurrencias: 62
Enfoque de unica recurrencia: 62


In [2]:
from ProblemaP1Tests import *
from FullDP import *

A =  [
        [0,9,1,10,0],
        [-1,-1,5,-1,5],
        [1,5,1,5,7],
        [5,5,5,15,2],
        [55,3,0,4,1]
    ]

print(f'Enfoque de varias recurrencias: {solvePathsProblem(A)}')
print(f'Enfoque de unica recurrencia: {fullSolutionDP(A)[0]}')

Enfoque de varias recurrencias: 32
Enfoque de unica recurrencia: 32


Probamos ejemplos pequeños y verificables

Probamos con un caso soluble de 5x5

In [3]:
from ProblemaP1Tests import *
from FullDP import *

A = [
        [0,-1,-1,-1,0],
        [2,3,-1,5,4],
        [1,5,15,2,7],
        [-1,1,-1,2,-1],
        [-1,-1,0,-1,-1]
    ]
print(f'Enfoque de varias recurrencias: {solvePathsProblem(A)}')
print(f'Enfoque de unica recurrencia: {fullSolutionDP(A)[0]}')

Enfoque de varias recurrencias: 37
Enfoque de unica recurrencia: 37


Probamos con el caso trivial más pequeño

In [4]:
from ProblemaP1Tests import *
from FullDP import *

A = [
        [0,0,0]
    ]
print(f'Enfoque de varias recurrencias: {solvePathsProblem(A)}')
print(f'Enfoque de unica recurrencia: {fullSolutionDP(A)[0]}')

Enfoque de varias recurrencias: 0
Enfoque de unica recurrencia: 0


Probamos un camino donde ningun personaje salga con vida

In [5]:
from ProblemaP1Tests import *
from FullDP import *

A = [
        [0,-1,-1,-1,0],
        [-1,-1,-1,-1,-1],
        [1,5,15,2,7],
        [-1,-1,-1,-1,-1],
        [-1,-1,0,-1,-1]
    ]

print(f'Enfoque de varias recurrencias: {solvePathsProblem(A)}')
print(f'Enfoque de unica recurrencia: {fullSolutionDP(A)[0]}')

Enfoque de varias recurrencias: -1
Enfoque de unica recurrencia: -1


Probamos el rendimiento de la solución

In [137]:
import random
from ProblemaP1Tests import *
from FullDP import *

def generar_matriz_aleatoria_impar(min_lado=3, max_lado=200):
    # Asegurarse de que el mínimo y máximo son impares y están en el rango correcto
    min_lado = min_lado if min_lado % 2 != 0 else min_lado + 1
    max_lado = max_lado if max_lado % 2 != 0 else max_lado - 1
    
    # Generar un tamaño impar aleatorio entre min_lado y max_lado
    a = random.choice(range(min_lado, max_lado + 1, 2))
    b = random.choice(range(min_lado, max_lado + 1, 2))
    
    # Crear la matriz con valores aleatorios (-1 o entre 1 y 1000)
    m = [[random.choice([-1, random.randint(1, 1000)]) for _ in range(b)] for _ in range(a)]
    
    m[0][0] = 0                       # Esquina superior izquierda
    m[0][b-1] = 0                     # Esquina superior derecha
    m[a-1][b//2] = 0                  # Centro de la última fila
    
    return m

In [7]:
matrices = [generar_matriz_aleatoria_impar() for _ in range(100)]

In [8]:
%%timeit
for m in matrices:
    solvePathsProblem(m)

5.07 s ± 342 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Probamos el rendimiento con matrices del tamaño más grande posible

In [9]:
matrices = [generar_matriz_aleatoria_impar(min_lado=199) for _ in range(100)]

In [10]:
%%timeit
for m in matrices:
    solvePathsProblem(m)

18.5 s ± 2.13 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


Probamos la entrada y salida estandar

In [11]:
filename = 'P1.in'
with open(filename, 'r') as file:
    content = file.read()
print(content)

2
5 5
0 9 1 10 0
-1 5 5 25 5
1 5 1 5 7
5 5 5 15 2
55 3 0 4 1
5 5
0 9 1 10 0
-1 -1 5 -1 5
1 5 1 5 7
5 5 5 15 2
55 3 0 4 1


In [12]:
!python ProblemaP1.py < P1.in > P1_python.out

In [13]:
filename = 'P1_python.out'
with open(filename, 'r') as file:
    content = file.read()
print(content)

62
32



Verificamos la solucion usando ejemplos no tan grandes aleatorios usando en comparativa la recurrencia completa

In [138]:
n = 150
diff = []
matrices = [generar_matriz_aleatoria_impar(max_lado=5) for _ in range(n)]
for m in matrices:
    n1 = solvePathsProblem(m)
    n2 = fullSolutionDP(m)[0]
    if(n1!=n2):
        print('Differences found')
        diff.append(m)

Differences found


In [140]:
for row in diff[0]:
    print(row)

[0, 784, 0]
[-1, 941, 695]
[236, 319, 478]
[-1, -1, 150]
[-1, 0, 154]


In [150]:
from ProblemaP1Tests import *
from FullDP import *

A = [[0, 784, 0],
[-1, 941, 695],
[236, 319, 478],
[-1, -1, 150],
[-1, 0, 154]]


fullSolutionDP(A)[0] , solvePathsProblem(A)

(2819, 2583)

In [152]:
for row in A: print(row)

[0, 784, 0]
[-1, 941, 695]
[236, 319, 478]
[-1, -1, 150]
[-1, 0, 154]


Dado que se identifico un contraejemplo para el metodo, se utilizara unica y exclusivamente la solución de programación dinamica pura. Adicionalmente se implementara en Java para contrarrestar la ineficiencia de las listas dinamicas de Python, se implementara en Java usando arreglos

In [41]:
filename = 'P1.in'
with open(filename, 'r') as file:
    content = file.read()
print(content)

2
5 5
0 9 1 10 0
-1 5 5 25 5
1 5 1 5 7
5 5 5 15 2
55 3 0 4 1
5 5
0 9 1 10 0
-1 -1 5 -1 5
1 5 1 5 7
5 5 5 15 2
55 3 0 4 1


In [42]:
!java ProblemaP1.java < P1.in > P1_java.out

In [43]:
filename = 'P1_java.out'
with open(filename, 'r') as file:
    content = file.read()
print(content)

62
32



In [50]:
!java ProblemaP1.java < P1_costume.in

62
32
0
0
-1
2819


Probamos el rendimiento general del algoritmo con numerosos casos del tamaño más grande posible

In [45]:
import random

def generar_matriz_aleatoria_impar(min_lado=3, max_lado=200):
    # Asegurarse de que el mínimo y máximo son impares y están en el rango correcto
    min_lado = min_lado if min_lado % 2 != 0 else min_lado + 1
    max_lado = max_lado if max_lado % 2 != 0 else max_lado - 1
    
    # Generar un tamaño impar aleatorio entre min_lado y max_lado
    a = random.choice(range(min_lado, max_lado + 1, 2))
    b = random.choice(range(min_lado, max_lado + 1, 2))
    
    # Crear la matriz con valores aleatorios (-1 o entre 1 y 1000)
    m = [[random.choice([-1, random.randint(1, 1000), random.randint(1, 1000), random.randint(1, 1000)]) for _ in range(b)] for _ in range(a)]
    
    m[0][0] = 0                       # Esquina superior izquierda
    m[0][b-1] = 0                     # Esquina superior derecha
    m[a-1][b//2] = 0                  # Centro de la última fila
    
    return m

def save_matrix_to_file(matrixes, filename):
    with open(filename, 'w') as file:
        n = len(matrixes)
        file.write(f"{n}\n")
        for matrix in matrixes:
            rows = len(matrix)
            cols = len(matrix[0]) if rows > 0 else 0
            file.write(f"{rows} {cols}\n")
            for row in matrix:
                file.write(" ".join(map(str, row)) + "\n")

In [46]:
n = 100
m = [generar_matriz_aleatoria_impar(min_lado=199) for _ in range(n)]
save_matrix_to_file(m, 'P1_BigInput.in')

In [47]:
%%timeit

!java ProblemaP1.java < P1_BigInput.in > P1_BigInputjava.out

2.22 s ± 41.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Probamos un conjunto de 500 entradas aleatorias y revisamos el desempeño obtenido

In [48]:
n = 500
m = [generar_matriz_aleatoria_impar() for _ in range(n)]
save_matrix_to_file(m, 'P1_RandomPerformance.in')

In [49]:
%%timeit

!java ProblemaP1.java < P1_RandomPerformance.in > P1_RandomPerformancejava.out

7.94 s ± 133 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
